In [1]:
import numpy as np
import pandas as pd
from scipy import io as sio

# 1 Data Preparation
### 1.1 Read Data

In [2]:
data = sio.loadmat("./ex8/ex8_movies.mat")
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'Y', 'R'])

In [3]:
Y, R = data["Y"], data["R"] #Y-> all moive ratings R-> if rated
Y.shape, R.shape

#1682 moives and 943 users

((1682, 943), (1682, 943))

In [4]:
param_mat = sio.loadmat('./ex8/ex8_movieParams.mat')
param_mat.keys()

dict_keys(['__header__', '__version__', '__globals__', 'X', 'Theta', 'num_users', 'num_movies', 'num_features'])

In [5]:
X, Theta, nu, nm, nf= param_mat["X"], param_mat["Theta"], param_mat["num_users"], param_mat["num_movies"], param_mat["num_features"]

In [6]:
X.shape, Theta.shape, nu, nm,nf

((1682, 10),
 (943, 10),
 array([[943]], dtype=uint16),
 array([[1682]], dtype=uint16),
 array([[10]], dtype=uint8))

In [7]:
#save as int dtype
nu, nm, nf = map(int,[nu, nm, nf])
nu, nm, nf

(943, 1682, 10)

# 2. Model Implementation

### 2.1 Serialization
The reason of using serialization is because we need to use scipy to minimize both and Theta simultaneously. However, Scipy requires 1-d array only.

In [8]:
def serialize(X, Theta):
    return np.append(X.flatten(), Theta.flatten())

### 2.2 Deserialization

In [9]:
def deserialize(params, nm, nu, nf):
    X = params[:nm*nf].reshape(nm,nf)
    Theta = params[nm*nf:].reshape(nu, nf)
    return X, Theta

### 2.3 Cost Function

In [10]:
def costFunction(params, Y, R, nm, nu, nf, Lambda):
    X, Theta = deserialize(params,nm, nu, nf)
    error = 0.5 * np.square((X @Theta.T- Y) * R).sum() # rated only, R(- {0,1}
    reg1 = 0.5 * Lambda * np.square(X).sum()
    reg2 = 0.5 * Lambda * np.square(Theta).sum()
    return error + reg1 + reg2

In [11]:
#slice for testing
users = 4
moives = 5
features = 3
X_sub = X[:moives,:features]
Theta_sub = Theta[:users,:features]
Y_sub = Y[:moives,:users]
R_sub = R[:moives,:users]
cost1 = costFunction(serialize(X_sub, Theta_sub), Y_sub, R_sub, moives, users, features, Lambda = 0)
cost1

22.224603725685675

In [12]:
cost2 = costFunction(serialize(X_sub, Theta_sub), Y_sub, R_sub, moives, users, features, Lambda = 1.5)
cost2

31.344056244274217

### 2.4 Gradient

In [37]:
def costGradient(params, Y,R, nm, nu, nf, Lambda):
    X, Theta = deserialize(params, nm, nu, nf)
    X_grad = ((X @ Theta.T - Y) * R) @ Theta + Lambda * X
    Theta_grad =((X @ Theta.T - Y) * R).T @ X + Lambda * Theta
    return serialize(X_grad,Theta_grad)

In [38]:
# add a new user
my_ratings = np.zeros((nm,1))
my_ratings[9] = 5
my_ratings[66] = 5
my_ratings[96] = 5
my_ratings[121] = 4
my_ratings[148] = 4
my_ratings[285] = 3
my_ratings[490] = 4
my_ratings[599] = 4
my_ratings[643] = 4
my_ratings[958] = 5
my_ratings[1117]= 3

In [39]:
# add a column
Y = np.c_[Y, my_ratings]
R = np.c_[R, my_ratings!= 0]
nm,nu = Y.shape

### 2.5 Mean Normalization

In [40]:
def normalizeRatings(Y,R):
    Y_mean = (Y.sum(axis =1)/ R.sum(axis = 1)).reshape(-1,1)
    Y_norm = (Y-Y_mean)* R
    return Y_norm, Y_mean

Y_norm, Y_mean = normalizeRatings(Y,R)

### 2.6 Parameters Initialization

In [41]:
X = np.random.random((nm,nf))
Theta = np.random.random((nu,nf))
params = serialize(X,Theta)

Lambda = 5

### 2.6 Gradient Descent with Scipy

In [43]:
from scipy.optimize import minimize
res = Y_minimize(fun = costFunction,
         x0 = params,
         args = (Y_norm, R, nm, nu, nf, Lambda),
         method = "TNC",
         jac = costGradient,
         options={"maxiter": 100})
params_fit = res.x
fit_X, fit_Theta =deserialize(params_fit,nm,nu,nf)

# 3. Model Evaluation

In [47]:
#all user
Y_pred = fit_X @ fit_Theta.T

In [61]:
# new added user
y_pred =  Y_pred[:,-1] + Y_mean.flatten()
# np.argsort -> sort values in ascending order and return each index, use [::-1] or np.argsort(-y_pred) to reverse 
index = np.argsort(y_pred)[::-1] 
# top 10 moive recommended
index[:10]

array([1598, 1121,  813, 1535, 1188, 1292, 1652, 1200, 1499, 1466],
      dtype=int64)

In [68]:
movies = []
with open("./ex8/movie_ids.txt", "r",encoding = "latin 1") as f:
    for line in f:
        tokens = line.strip().split()
        movies.append("".join(tokens[1:]))
for i in range(10):
    print(index[i], movies[index[i]], y_pred[index[i]])

1598 SomeoneElse'sAmerica(1995) 5.003268209470818
1121 TheyMadeMeaCriminal(1939) 5.001885788673767
813 GreatDayinHarlem,A(1994) 5.001714194980393
1535 Aiqingwansui(1994) 5.000859107481553
1188 Prefontaine(1997) 5.00063688523736
1292 StarKid(1997) 4.999864592553755
1652 EntertainingAngels:TheDorothyDayStory(1996) 4.999828299043169
1200 MarleneDietrich:ShadowandLight(1996) 4.99859579231539
1499 SantawithMuscles(1996) 4.998493754110596
1466 SaintofFortWashington,The(1993) 4.996387203463531


Now, we have predicted 10 movies for the new user we added :)